# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = "train.p"
validation_file= "valid.p"
testing_file = "test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# First make sure the volume of data is correct
assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))


n_train = len(y_train)
n_validation = len(y_valid)
n_test = len(y_test)

image_shape = X_train[0].shape
n_classes = len(set(y_train))

print("Number of training examples = ", n_train)
print("Number of validation examples = ", n_validation)
print("Number of testing examples = ", n_test)
print("Image data shape = ", image_shape)
print("Number of classes = ", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
import matplotlib.pyplot as plt

# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
# Look at the distribution of classes in the training, validation and test set
# => result: found that the data is imbalanced

from collections import Counter
import numpy as np

print("Distribution of Training set:")

train_label_classes, num_of_train_labels = zip(*Counter(y_train).items())

indexes = np.arange(len(train_label_classes))
width = 1

plt.bar(indexes, num_of_train_labels, width)
plt.show()

print("Distribution of Validation set:")

valid_label_classes, num_of_valid_labels = zip(*Counter(y_valid).items())
plt.bar(indexes, num_of_valid_labels, width)
plt.show()

print("Distribution of Test set:")
test_label_classes, num_of_test_labels = zip(*Counter(y_test).items())
plt.bar(indexes, num_of_test_labels, width)
plt.show()

In [ ]:
# plot a random image
import random


index = random.randint(0, len(X_train))

print("random index: ", index)
print("image original size: ", train['sizes'][index][0], 'x', train['sizes'][index][1])
print("sign position in original image: ", train['coords'][index])

print("type of single image in X_train: ", type(X_train[index]))
print("type of single pixel intensity of one image in X_train: ", X_train[index].dtype)

coords = train['coords'][index]

# Can't use mpimg.imread directly read image from numpy.ndarray
# -> TypeError: Object does not appear to be a 8-bit string path or a Python file-like object
# ex. image = mpimg.imread(X_train[10000])

plt.figure(figsize=(1,1))
# interpolation='nearest' simply display the image without try to interpolate betwen pixels 
# if the display resolution is not the same as the image resolution (which is most often the case)
plt.imshow(X_train[index], interpolation='nearest')
print()
print("label of the picture: ", y_train[index])

In [ ]:
# plot a bunch of random images from all 43 classifications

# get the idea from the blog of Alex Staravoitau
# https://navoshta.com/traffic-signs-classification/

from pandas.io.parsers import read_csv

signnames = read_csv("signnames.csv").values[:, 1]
sign_classes, class_indices, class_counts = np.unique(y_train, return_index = True, return_counts = True)

col_width = max(len(name) for name in signnames)

for sign_class, sign_index, sign_count in zip(sign_classes, class_indices, class_counts):
    print("Class {}: {:<{}}  {} samples".format(sign_class, signnames[sign_class], col_width, sign_count))
    fig = plt.figure(figsize = (10, 1))
    random_indices = random.sample(range(sign_index, sign_index + sign_count), 10)
    for i in range(10):
        axis = fig.add_subplot(1, 10, i + 1, xticks=[], yticks=[])
        axis.imshow(X_train[random_indices[i]])
    plt.show()
    print("--------------------------------------------------------------------------------------\n")

From above, we know that the dataset we use 
(1) is imbalanced
(2) has images that differ significantly in terms of contrast and brightness

In [ ]:
Because of such an imbalanced data, the great accurancy skew in favor of few classes, 
and the result turned out to be a lie.

According to "8 Tactics to Combat Imbalanced Classes in Your Machine Learning Dataset", the article in the blog of  Dr. Jason Brownlee, 8 
http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
    
It may be the good idea to "get more data" to reduce imbalance.

But before doing things like rotating, shifting, it would be better to tackle brightness problem first.

In [ ]:
From the paper writed by "Pierre Sermanet and Yann LeCun", It tackle contrast problem with "converted to YUV space. 
The Y channel is then preprocessed with global and local contrast normalization
while U and V channels are left unchanged"

Also the article implicitly reveal that the result about using grayscale images to classify bacically eqauls to RGB images.


From this post, https://read01.com/0MOGzg.html. I got the idea that the importance of YUV colorspace is 
that it can seperate luminance signal (Y) and chrominance signal (U, V). So, if we only take Y component, the image convert 
to grayscale, and fortunately that is what Opencv function cv2.RGB2GRAY() does for us.

YUV色彩空間的重要性是它的亮度信號Y和色度信號U、V是分離的
    
如果只有Y信號分量而沒有U、V分量， 那麼這樣表示的圖就是黑白灰度圖。
原文網址：https://read01.com/0MOGzg.html
    
From Wiki of YUV
Y = 0.299 * R + 0.587 * G + 0.114 * B
U = 0.492 * (B - Y)
V = 0.877 * (R - Y)

Then after converting to grayscale.

https://stackoverflow.com/questions/10684220/manipulating-luma-in-yuv-color-space
http://docs.opencv.org/trunk/d3/dc1/tutorial_basic_linear_transform.html

We can just do linear transformation on Y to change brightness and contrast

new Y value = contrast(default = 1.0) * old_Y + brightness(default = 0.0)

In [ ]:
import cv2

# convert to grayscale
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


X_train_gray = []
X_valid_gray = []
X_test_gray = []

for img in X_train:
    X_train_gray.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))

for img in X_valid:
    X_valid_gray.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    
for img in X_test:
    X_test_gray.append(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))

In [ ]:
X_train = np.array(X_train_gray).reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_valid = np.array(X_valid_gray).reshape(X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], 1)
X_test = np.array(X_test_gray).reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [ ]:
from pandas.io.parsers import read_csv
import random

signnames = read_csv("signnames.csv").values[:, 1]
sign_classes, class_indices, class_counts = np.unique(y_train, return_index = True, return_counts = True)

col_width = max(len(name) for name in signnames)


for sign_class, sign_index, sign_count in zip(sign_classes, class_indices, class_counts):
    print("Class {}: {:<{}}  {} samples".format(sign_class, signnames[sign_class], col_width, sign_count))
    fig = plt.figure(figsize = (10, 1))
    random_indices = random.sample(range(sign_index, sign_index + sign_count), 10)
    for i in range(10):
        axis = fig.add_subplot(1, 10, i + 1, xticks=[], yticks=[])
        axis.imshow(X_train[random_indices[i]].squeeze(), cmap="gray")
    plt.show()
    print("--------------------------------------------------------------------------------------\n")

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.
print(X_train.dtype)
print(y_train.dtype)

In [ ]:
# # First we should change type of features from int8 to float32 in order to do normalize
X_train = X_train.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test = X_test.astype(np.float32)

# Then set flags for feature engineering tasks I am going to do.  This will prevent me from skipping an important step.
is_features_normal = False
# is_labels_encod = False

In [ ]:
def normalize_dataset(image_data):
    """
    Normalize the image data
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """    
    # +, -, *, / in np.array will automatically broadcast to every element
    return (image_data - 128) / 128

# Test Cases
np.testing.assert_array_almost_equal(
    normalize_dataset(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 255])),
    [-1., -0.9921875, -0.984375 , -0.9765625, -0.96875  ,-0.9609375, -0.953125 , -0.9453125, -0.9375, 
     -0.9296875, -0.921875 ,  0.9921875], 
    decimal=3)

if not is_features_normal:
    print("Normalizing ...")
    X_train = normalize_dataset(X_train)
    X_valid = normalize_dataset(X_valid)
    X_test = normalize_dataset(X_test)
    is_features_normal = True

print("Finish Normalization !")

In [ ]:
print(X_train[0])

In [ ]:
# from sklearn.preprocessing import LabelBinarizer

# if not is_labels_encod:
#     print('One-Hot Encoding ...')
#     # Turn labels into numbers and apply One-Hot Encoding
#     encoder = LabelBinarizer()
#     encoder.fit(y_train)
#     y_train = encoder.transform(y_train)
#     y_valid = encoder.transform(y_valid)
#     y_test = encoder.transform(y_test)
    
#     # Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
#     y_train = y_train.astype(np.float32)
#     y_valid = y_valid.astype(np.float32)
#     y_test = y_test.astype(np.float32)
#     is_labels_encod = True


# print('Finish labels One-Hot Encoded')

In [ ]:
# Make sure we do preprcess before training the model

assert is_features_normal, 'You skipped the step to normalize the features'
# assert is_labels_encod, 'You skipped the step to One-Hot Encode the labels'

In [ ]:
# Before store the preprocessing data to one pickle, shuffle the data

from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

import os

# Save the data after preprocessing step for easy access
pickle_file = 'traffic.p'
# if not os.path.isfile(pickle_file):
print('Saving data to pickle file...')
try:
    with open(pickle_file, 'wb') as pfile:
        pickle.dump(
            {
                'X_train': X_train,
                'y_train': y_train,
                'X_valid': X_valid,
                'y_valid': y_valid,
                'X_test': X_test,
                'y_test': y_test,
            },
            pfile, pickle.HIGHEST_PROTOCOL)
except Exception as e:
    print('Unable to save data to', pickle_file, ":", e)
    raise

print('Data cached in pickle file.')

### Reload Pickle data

In [1]:
%matplotlib inline

# Load the modules
import pickle
import math

import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

# Reload the data
pickle_file = 'traffic.p'
with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  X_train = pickle_data['X_train']
  y_train = pickle_data['y_train']
  X_valid = pickle_data['X_valid']
  y_valid = pickle_data['y_valid']
  X_test = pickle_data['X_test']
  y_test = pickle_data['y_test']
  del pickle_data  # Free up memory


print('Data and modules loaded.')

Data and modules loaded.


In [15]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# First make sure the volume of data is correct
assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))


n_train = len(y_train)
n_validation = len(y_valid)
n_test = len(y_test)

image_shape = X_train[0].shape
n_classes = len(set(y_train))

print("Number of training examples = ", n_train)
print("Number of validation examples = ", n_validation)
print("Number of testing examples = ", n_test)
print("Image data shape = ", image_shape)
print("Number of classes = ", n_classes)

Number of training examples =  34799
Number of validation examples =  4410
Number of testing examples =  12630
Image data shape =  (32, 32, 1)
Number of classes =  43


In [3]:
print(X_train.dtype)
print(y_train.dtype)

float32
uint8


### Model Architecture

In [4]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

import tensorflow as tf

EPOCHS = 10
BATCH_SIZE = 128

In [18]:
from tensorflow.contrib.layers import flatten

def LeNet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1    
    
    # P.s strides = [batch, height, width, depth]
    
    # TODO: Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 6), mean = mu, stddev = sigma)) # (height, width, input_depth, output_depth)
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b 
    
    # TODO: Activation.
    conv1 = tf.nn.relu(conv1)

    # TODO: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    # add Dropout
    conv1 = tf.nn.dropout(conv1, keep_prob)
    
    # TODO: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # TODO: Activation.
    conv2 = tf.nn.relu(conv2)

    # TODO: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    # add Dropout    
    conv2 = tf.nn.dropout(conv2, keep_prob)
    
    # TODO: Flatten. Input = 5x5x16. Output = 400.
    fc0 = flatten(conv2)
    
    # TODO: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400,120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc0, fc1_W) + fc1_b
    
    # TODO: Activation.
    fc1 = tf.nn.relu(fc1)

    # add Dropout    
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    # TODO: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1, fc2_W) + fc2_b
    
    # TODO: Activation.
    fc2 = tf.nn.relu(fc2)
    
    # TODO: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W = tf.Variable(tf.truncated_normal(shape=(84, 43), mean = mu, stddev = sigma))
    fc3_b = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [12]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.uint8, (None))
one_hot_y = tf.one_hot(y, 43)
keep_prob = tf.placeholder(tf.float32)

rate = 0.001

logits = LeNet(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [13]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [20]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
            
        
        train_accuracy = evaluate(X_train, y_train)
        print("EPOCH {} ...".format(i+1))
        print("Train Accuracy = {:.3f}".format(train_accuracy))
        print("Training Error = {}".format(sess.run(loss_operation, feed_dict={x: X_train, y: y_train, keep_prob: 1.0})))
        print()
        
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Cross Validation Error = {}".format(sess.run(loss_operation, feed_dict={x: X_valid, y: y_valid, keep_prob: 1.0})))
        print()
        
    saver.save(sess, 'record/lenet')
    print("Model saved")

Training...

EPOCH 1 ...
Train Accuracy = 0.800
Training Error = 0.7187942862510681

EPOCH 1 ...
Validation Accuracy = 0.731
Cross Validation Error = 0.9355849623680115

EPOCH 2 ...
Train Accuracy = 0.911
Training Error = 0.32824093103408813

EPOCH 2 ...
Validation Accuracy = 0.838
Cross Validation Error = 0.5802712440490723

EPOCH 3 ...
Train Accuracy = 0.952
Training Error = 0.19968441128730774

EPOCH 3 ...
Validation Accuracy = 0.878
Cross Validation Error = 0.4628375470638275

EPOCH 4 ...
Train Accuracy = 0.963
Training Error = 0.1497156023979187

EPOCH 4 ...
Validation Accuracy = 0.876
Cross Validation Error = 0.41939252614974976

EPOCH 5 ...
Train Accuracy = 0.973
Training Error = 0.10773298144340515

EPOCH 5 ...
Validation Accuracy = 0.893
Cross Validation Error = 0.4116692841053009

EPOCH 6 ...
Train Accuracy = 0.982
Training Error = 0.0780087411403656

EPOCH 6 ...
Validation Accuracy = 0.917
Cross Validation Error = 0.33553963899612427

EPOCH 7 ...
Train Accuracy = 0.983
Train

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")